In [6]:
def Mon(I):
    Rold=I.ring()
    n=Rold.ngens()
    char=str(Rold.gens()[0])[0]
    
    for k in range(1,2*n+1):
        var(char+str(k))
    
    Rbig=PolynomialRing(QQ,[(char+'%s') %k for k in range(1,2*n+1)],order='invlex')
    R=PolynomialRing(QQ,[(char+'%s') %k for k in range(1,n+1)],order='lex')
    GB=I.groebner_basis()
    
    m=Rbig(1)
    for k in range(n+1,2*n+1):
        m*=Rbig(char+str(k))
    GB_big=[m-1]
 
    for f in GB:
        f=Rbig(f)
        for k in range(1,n+1):
            f=f.subs({Rbig(char+str(k)):Rbig(char+str(k))*Rbig(char+str(n+k))})
        GB_big+=[f]

    MonI=[]
    GB_sol=ideal(Rbig,GB_big).groebner_basis()
    print(GB_sol)
    for g in GB_sol:
        if g.lm() < Rbig(char+str(n+1)) and g.lt()==g:
            MonI+=[g]
    return Ideal(R,MonI).groebner_basis()

In [7]:
import numpy as np
def BinO(I,t):
    n=I.ring().ngens()
    Rold=I.ring()
    R.<x,y,z,w> = PolynomialRing(QQ,4,order="deglex")
    if(n>1):
        I=I.change_ring(R).groebner_basis()
    else:
        I=I.groebner_basis()
    p=[]
    if(n==4):
        for f in I:
            v=f.degree()
            for a in range(t-v+1):
                for b in range(t-v-a+1):
                    for c in range(t-v-a-b+1):
                        for d in range(t-v-a-b-c+1):
                            p.append(f*x^a*y^b*z^c*w^d)
    elif(n==3):
        for f in I:
            v=f.degree()
            for a in range(t-v+1):
                for b in range(t-v-a+1):
                    for c in range(t-v-a-b+1):
                        p.append(f*x^a*y^b*z^c)
    elif(n==2):
        for f in I:
            v=f.degree()
            for a in range(t-v+1):
                for b in range(t-v-a+1):
                        p.append(f*x^a*y^b)
    elif(n==1):
        for f in I:
            for a in range(t-f.degree()+1):
                        p.append(f*x^a)              
                    
    A=Sequence(p).coefficient_matrix()
    M=Matroid(A[0].right_kernel().basis_matrix()).circuits()
    X=[]
    for c in M:
        if len(c)==2:
            x=list(c)
            m=R(A[1][x[0]])
            n=R(A[1][x[1]])
            if m>n:
                m,n=n,m
            X.append(n-m*(n.reduce(I)/m.reduce(I)))
    I=ideal(R,ideal(R,X).groebner_basis()).change_ring(Rold)
    return I.groebner_basis()

In [8]:
import numpy as np
def Bin(I,t):
    Rold=I.ring()
    n=Rold.ngens()
    char=str(I.ring().gens()[0])[0]
    R=PolynomialRing(QQ,[(char+'%s') %k for k in range(1,n+1)],order='deglex')
    Pall=[]
    GBI=I.groebner_basis()
  

    #Setting up all the vectors
    P=GBI
    for x in Rold.gens():
        xToP=[]
        for g in P:
            v=g.degree()
            for k in range(t-v+1):
                xToP.append(g*x^k)
        P=xToP
    A=Sequence(ideal(P)).coefficient_matrix()
    M=Matroid(A[0].right_kernel().basis_matrix()).circuits()
    X=[]
    for c in M:
        if len(c)==2:
            print(c)
            s=list(c)
            m=R(A[1][s[0]])
            n=R(A[1][s[1]])
            if m>n:
                m,n=n,m
            X.append(n-m*(n.reduce(I)/m.reduce(I)))
    BinI=ideal(R,ideal(R,X).groebner_basis()).groebner_basis()
    return BinI

In [9]:
def ugb(I):
    R=I.ring()
    Sol=[]
    V=I.groebner_fan().reduced_groebner_bases()
    for v in V:
        for g in v:
            add=True
            for f in Sol:
                if (R(g)/R(f)) in QQ:
                    add=False
            if(add):
                Sol.append(g)
        Sol.sort()
    return Sol

In [2]:
def getJ(I,V):
    n=I.ring().ngens()
    d=len(V)
    char=str(I.ring().gens()[0])[0]
    Main=I.groebner_basis()
    for k in range(1,n+d+1):
        var(char+str(k))
    L=LaurentPolynomialRing(QQ,[(char+'%s') %k for k in range(1,n+d+1)],order='lex') 
    Rbig=PolynomialRing(QQ,[(char+'%s') %k for k in range(1,n+d+1)],order='lex') 
    R=   PolynomialRing(QQ,[(char+'%s') %k for k in range(n+1,n+d+1)],order='deglex') 
    RofJ=PolynomialRing(QQ,[(char+'%s') %k for k in range(1,d+1)],order='deglex') 
    
    #go to R[A]
    sub=[]
    for v in range(d):
        mon=L(1)
        for e in range(n):
            mon*=(L(char+str(e+1))^(V[v][e]))
        Main+=[mon-L(char+str(v+1+n))]
        sub.append(mon)
    GB=ideal(L,Main).groebner_basis()
    
    #Find J
    genJhigh=[]
    for g in GB:
        if Rbig(g).lm()< L(char+str(n)):
            genJhigh.append(Rbig(g))
    genJ=[]
    for f in genJhigh:
        for k in range(1,d+1):
            f=f.subs({Rbig(char+str(n+k)):Rbig(char+str(k))})
        genJ.append(f)
    J=ideal(RofJ,genJ)
    return J.groebner_basis()

In [26]:
R.<a1,a2>=QQ[]
n=13
I=ideal(R,[a1^2-a2,a2^3-2])
[A,B]=getM(I)
B.det()

[1, a1, a2, a1*a2, a2^2, a1*a2^2]


4

In [22]:
print(log(A))

[ 1/12  1/12]
[-1/12 -1/12]


In [6]:
A.charpoly().factor()

(x^2 + 1) * (x^2 + x + 1)

In [63]:
A=matrix(QQ,[[1,2,0,-1,5],[2,0,2,0,1],[1,1,-1,3,2],[0,3,-3,2,6]]).T
B=matrix(QQ,[[1,2,3],[2,1,1],[0,2,1]])
Bp=matrix(QQ,[[0,1,2],[1,0,1],[0,1,0]])
G=matrix(QQ,[[3,1,1],[0,-1,-3],[0,-5,1]])
Bp^-1*B

[3/2   1   0]
[  0   2   1]
[1/2   0   1]

In [69]:
kern=span([ vector(QQ,[-7,0,6,3,2]),vector(QQ,[0,5,1,0,-2])]  )
vector(QQ,[-7,0,6,3,2]) in kern

True

In [60]:
Im=span([vector(QQ,[1,2,1,0]),vector(QQ,[2,0,1,3]),vector(QQ,[0,-2,1,3])])
vector(QQ,[0,0,1,1]) in Im

True

In [79]:
(a^12-1)/(a^4-a^3+2*a^2-a+1)

a^8 + a^7 - a^6 - 2*a^5 + 2*a^3 + a^2 - a - 1

In [80]:
(a^12-1)/(a^4+a^3+2*a^2+a+1)

a^8 - a^7 - a^6 + 2*a^5 - 2*a^3 + a^2 + a - 1

In [81]:
(a^8 - a^7 - a^6 + 2*a^5 - 2*a^3 + a^2 + a - 1).factor()

(a - 1) * (a + 1) * (a^2 - a + 1) * (a^4 - a^2 + 1)

In [87]:
(a^4 - a^3 + 2*a^2 - a + 1)*(a^8 - a^7 - a^6 + 2*a^5 - 2*a^3 + a^2 + a - 1)

a^12 - 2*a^11 + 2*a^10 - 2*a^8 + 2*a^7 - 2*a^5 + 2*a^4 - 2*a^2 + 2*a - 1

In [1]:
#Input: I and T(I)^l
#Output: |T(I)^l| matrices M 
def getM(J):
    J=J.groebner_basis()
    d=J.ring().ngens()
    char=str(I.ring().gens()[0])[0]
     
    R=PolynomialRing(QQ,[(char+'%s') %k for k in range(1,d+1)],order='lex')

    #Basis of R/(J)
    Basis=[R(1)]; 
    HeadJ=[]

    for g in J:
        HeadJ.append(g.lm())
   
    if(d==1):
        Basis=[R(char+'1')^k for k in range(J[0].degree())]
    else:
        for x in R.gens():
            for b in Basis:
                if not (b*x).reduce(HeadJ)==0:
                    Basis.append(b*x)
    print(Basis)
    #makeMatrix
    bas=len(Basis)
    M=[]
    for x in R.gens():
        mset=[]
        for b in Basis:
            if(d==1):
                f=(b*x).mod(J[0]) 
            else:
                f=(b*x).reduce(J)
            dic=f.dict()
            row=bas*[0]
            for e in f.exponents():
                row[Basis.index(R({e:1}))]=dic[e]
            mset.append(row)
        M.append(matrix(QQ,mset).T)
    return M

In [15]:
def exp(N):
    n=len(N[0])
    if not (N^n).is_zero():
        raise ValueError('N non nilpotent')
    B=1
    for i in range(1,n+2):
        B=B+(1/(gamma(i+1)))*(N^i)
    return B
def log(U):
    n=len(U[0])
    if( not( (U-1)^n).is_zero()):
        raise ValueError('U non unipotent')
    B=0
    for i in range(1,n+2):
        B=B-(1/i)*((1-U)^i)
    return B

In [13]:
def Unip(U):
    Mset=[]
    for u in U:
        Mset.append(log(u).list())
    return matrix(QQ,Mset).kernel().basis()

In [14]:
#Input: K M in K
#Output: L1 and M' diagon
def split1(M,K):
    Diago=[]
    Mbig=[]
    for m in M:
        J=m.jordan_form(K)
        C = m.is_similar(J, transformation=True)[1]
        Cinv=C^(-1)
        #M=Cinv*J*(C)
        D=m.eigenmatrix_right()[0]
        Diago.append(Cinv*D*C)
        Mbig.append(log(m*Cinv*(D^-1)*C).list())
    return Mbig.kernel().basis(),Diago


#Returns U; M=UD=DU, U unipot & D diag.
def getD(M):
    S.<s>=QQ[]
    charA=S(M.characteristic_polynomial('s'))
    minpD = S(charA/(charA.gcd(charA.derivative(s))))#.derivative(s)))
    inv  = S(minpD.derivative(s)).xgcd(minpD)[1]
    Dbefore=zero_matrix(len(M[0]))
    D=M
    e=0
    print(charA)
    print(minpD)
    while not Dbefore==D:
        Dbefore,D=D,(D-minpD(D)*inv(D))
        e+=1
        print(D)
        if(e==100):
            break
    #return D  


In [ ]:
K.<polygen(QQ,'d'+str(1))>=K.extension(s^l-det2)
K.<polygen(QQ,'d'+str(1))>=K.extension(s^l-det3)
K=NumberField([x^2 +1], names='i')
K(-1).nth_root(2)
AA=matrix(K,A/(K(det(A)).nth_root(2)))
BB=matrix(K,B/(K(det(B)).nth_root(2)))
AA,BB
bas=2
K=NumberField([x^bas +1], names='a3')
KK.<s>=K[]
for k in range(5,8):
    if KK(s^bas-k).is_irreducible():
        K=K.extension(x^bas-k,name='a'+str(k))
        KK.<x>=K[]
K.absolute_vector_space()

In [ ]:
#Input: matrices M and splited determinants
#Output: Field K with all M/[l]root(det(M))
    Mbig=[]
    Gamma=[]   
    for m in M:
        J=m.jordan_form(subdivide=False)
        C = m.is_similar(J, transformation=True)[1]
        Cinv=C^(-1)
        #Cinv*J*(C)=M
        D=m.eigenmatrix_right()[0]
        Gamma.append(Cinv*D*C)
        #Mbig.append(log(M*Cinv*(D^-1)*C).list())

In [33]:
#Examples
S2.<x,y>=PolynomialRing(QQ,2, order="lex")
S3.<x,y,z>=PolynomialRing(QQ,3, order="lex")
T.<a,b>=PolynomialRing(QQ,2, order="lex")
I1n=[ideal(S2,[y-fibonacci(n)*x-fibonacci(n-1),x*x-x-1]) for n in range(20)]
I2n=[ideal(S3,[(x-z)^2,n*x-y-(n-1)*z]) for n in range(20)]
E1=ideal(S2,[x^3+2*x^2+2*x+1])
L=ideal(S2,[y-3*x-2,x*x-x-1])

In [449]:
e=matrix(QQ,[[0,1,0,0,0],
             [4,0,0,0,0],             
             [0,0,0,0,6/5],
             [0,0,0,-71,0],
             [0,0,326423,0,0]])

M=e*matrix(QQ,[[2421,1233121,12,1/7,13],
               [0,14,-7/8,-12342,142],
               [0,0,14,14,142],
               [0,0,0,14,142],
               [0,0,0,0,14]])*(e^-1)

J=M.jordan_form(subdivide=False)
C = M.is_similar(J, transformation=True)[1]
Cinv=C^(-1)
#(C^-1)*J*(C)=M
D=M.eigenmatrix_right()[0]
gamma=Cinv*D*C
U=M*Cinv*(D^-1)*C
gamma.jordan_form()

#M=(C^-1)*D*C*(  1  +(C^-1)*(D^-1)*N*C  )


[2421|   0|   0|   0|   0]
[----+----+----+----+----]
[   0|  14|   0|   0|   0]
[----+----+----+----+----]
[   0|   0|  14|   0|   0]
[----+----+----+----+----]
[   0|   0|   0|  14|   0]
[----+----+----+----+----]
[   0|   0|   0|   0|  14]